In [118]:
#Import Libraries

from pyspark.sql import SparkSession
from pyspark.sql.functions import *

In [119]:
#Configuration Jobs

spark = SparkSession \
    .builder \
    .appName("Flights") \
    .getOrCreate()

In [120]:
#Load csv Data to DataFrame

In [121]:
#Flights
flightsDF= spark.read.option("header","true").csv("./Data/flights.csv")
#Airports
airportsDF= spark.read.option("header","true").csv("./Data/airports.csv")
#Airline
airlinesDF= spark.read.option("header","true").csv("./Data/airlines.csv")

In [122]:
#Functions to transform or remove columns
def RemoveUnnecessaryColumnsFunction(df: DataFrame) -> DataFrame:
    columns_to_drop = [
                        'ORIGIN_AIRPORT',
                        'IATA_CODE',
                        'AIRPORT',
                        'CITY',
                        'STATE',
                        'COUNTRY',
                        'LATITUDE',
                        'LONGITUDE']
    return df.drop(*columns_to_drop)

In [123]:
#join airlines to flights to avoid code
flightsDF = flightsDF.join(airlinesDF,flightsDF.AIRLINE ==  airlinesDF.IATA_CODE,"inner").drop(flightsDF.AIRLINE).drop(col("IATA_CODE")).withColumnRenamed("AIRLINE","AIRLINE_") 
flightsDF = flightsDF.join(airportsDF,flightsDF.ORIGIN_AIRPORT ==  airportsDF.IATA_CODE,"inner").withColumnRenamed("CITY","CITY_DEPART_") 
flightsDF = RemoveUnnecessaryColumnsFunction(flightsDF)
flightsDF = flightsDF.join(airportsDF,flightsDF.DESTINATION_AIRPORT ==  airportsDF.IATA_CODE,"inner").withColumnRenamed("CITY","CITY_ARRIVAL_").drop(col("DESTINATION_AIRPORT")) 
flightsDF = RemoveUnnecessaryColumnsFunction(flightsDF)

In [124]:
#Transform Date

flightsDF = flightsDF.withColumn('DATE_', to_date(concat_ws('-',flightsDF.YEAR,flightsDF.MONTH,flightsDF.DAY))).drop(*['YEAR','MONTH','DAY','DAY_OF_WEEK'])
flightsDF = RemoveUnnecessaryColumnsFunction(flightsDF);
flightsDF.show(1);


+-------------+-----------+-------------------+--------------+---------------+--------+----------+--------------+------------+--------+--------+---------+-------+-----------------+------------+-------------+--------+---------+-------------------+----------------+--------------+-------------+-------------------+-------------+--------------------+------------+-------------+----------+
|FLIGHT_NUMBER|TAIL_NUMBER|SCHEDULED_DEPARTURE|DEPARTURE_TIME|DEPARTURE_DELAY|TAXI_OUT|WHEELS_OFF|SCHEDULED_TIME|ELAPSED_TIME|AIR_TIME|DISTANCE|WHEELS_ON|TAXI_IN|SCHEDULED_ARRIVAL|ARRIVAL_TIME|ARRIVAL_DELAY|DIVERTED|CANCELLED|CANCELLATION_REASON|AIR_SYSTEM_DELAY|SECURITY_DELAY|AIRLINE_DELAY|LATE_AIRCRAFT_DELAY|WEATHER_DELAY|            AIRLINE_|CITY_DEPART_|CITY_ARRIVAL_|     DATE_|
+-------------+-----------+-------------------+--------------+---------------+--------+----------+--------------+------------+--------+--------+---------+-------+-----------------+------------+-------------+--------+---------+--